In [1]:
import pandas as pd
import os
import re
import json
import gc

In [2]:
class AlphaBoosting:
    def __init__(self, root=None, train_csv_url=None, test_csv_url=None, validation_index=None, timestamp=None,
                 label=None, categorical_features=None, numerical_features=None, validation_ratio=0.1, ngram=(1,1),
                 downsampling=1, configuration=None):
        if configuration == None:
            # set configurations
            self.ROOT = root
            self.OUTDIR = root + 'output/'
            self.LOGDIR = root + 'log/'
            self.DATADIR = root + 'data/'
            self.train_csv_url = train_csv_url
            self.test_csv_url = test_csv_url
            self.timestamp = timestamp
            self.label = label
            self.categorical_features = categorical_features
            self.numerical_features = numerical_features
            self.downsampling = downsampling
            # read data
            self._read_data()
            if validation_index == None:
                self.validation_index = list(range(int(self.train_len*(1-validation_ratio)), self.train_len))
            else:
                self.validation_index = validation_index
        else:
            with open(configuration, 'r') as infile: file = json.load(infile)
            self.ROOT = file['root']
            self.OUTDIR = file['root'] + 'output/'
            self.LOGDIR = file['root'] + 'log/'
            self.DATADIR = file['root'] + 'data/'
            self.train_csv_url = file['train_csv_url']
            self.test_csv_url = file['test_csv_url']
            # read data
            self._read_data()
            self.timestamp = file['timestamp']
            self.label = file['label']
            self.categorical_features = file['categorical_features']
            self.numerical_features = file['numerical_features']
            self.validation_index = file['validation_index']
            self.downsampling = file['downsampling']
        
        # biuld relavent directories
        self.FEATUREDIR = self.DATADIR + 'features/'
        if not os.path.exists(self.OUTDIR): os.makedirs(self.OUTDIR)
        if not os.path.exists(self.LOGDIR): os.makedirs(self.LOGDIR)
        if not os.path.exists(self.DATADIR): os.makedirs(self.DATADIR)
        if not os.path.exists(self.FEATUREDIR): os.makedir(self.FEATUREDIR)
            
        # save configuration: c
        self._save_config(self.LOGDIR + 'config.json')
        
        # generate todo list: c
        dictionary = self._generate_todo_list()
        
        # feature engineering
        self._feature_engineering(dictionary)
        
        # get validation
        self._generate_validation(dictionary)
        
        # down sampling
        self._down_sampling(dictionary)
        
        # concatenant test: c
        self._concat_test(dictionary)
        
        # grid search
        self._grid_search(dictionary)
    
    
    ######### util functions #########
    def _read_data(self):
        self.train = pd.DataFrame()
        self.test = pd.DataFrame()
        if self.train_csv_url != None: self.train = pd.read_csv(self.train_csv_url)
        if self.test_csv_url != None: self.test = pd.read_csv(self.test_csv_url)
        self.df = pd.concat([self.train, self.test], ignore_index=True)
        self.train_len = self.train.shape[0]
        
    def _renew_status(self, dictionary, key, file_url):
        dictionary[key] = True
        with open(file_url, 'w') as file:
            json.dump(dictionary, file)
            
    def _save_config(self, url):
        d = {
            'root':                 self.ROOT,
            'train_csv_url':        self.train_csv_url,
            'test_csv_url':         self.test_csv_url,
            'timestamp':            self.timestamp,
            'label':                self.label,
            'categorical_features': self.categorical_features,
            'numerical_features':   self.numerical_features,
            'validation_index':     self.validation_index,
            'downsampling':         self.downsampling
        }
        with open(url, 'w') as file: json.dump(d, file)
        del d
        gc.collect()
            
    def _get_file_concat(self, split_folder, concat_folder, is_train, file_name_body, useindex=None):
        prefix = 'train' if is_train else 'test'
        if useindex == None:
            ftr = self.train.copy() if is_train else self.test.copy()
        else:
            ftr = self.train[useindex] if is_train else self.test[useindex]
        file_name = prefix + '__' + file_name_body + '.ftr'
        for file in os.path.listdirs(split_folder):
            inter_split = file.split('.')
            if inter_split[-1] == 'ftr':
                splitted = inter_split[0].split('__')
                if splitted[0] == prefix:
                    tmp_ftr = pd.read_feather(split_folder + file)
                    ftr['__'.join(splitted[1:])] = tmp_ftr if useindex == None else tmp_ftr[useindex]
                    del tmp_ftr
                    gc.collect()
        ftr.reset_index(drop=True).to_feather(concat_folder + file_name)
        del ftr
        gc.collect()
            
            
    
    ######### main functions #########
    def _generate_todo_list(self):
        if os.path.exists(self.LOGDIR + 'todo_list.json'):
            with open(self.LOGDIR + 'todo_list.json', 'r') as file:
                dictionary = json.load(file)
        else:
            dictionary = {'feature_engineering':           False, 
                          'val_downsample_generate_index': False,
                          'val_downsample_split':          (self.downsampling==0),
                          'val_downsample_generation':     False,
                          'concat_test':                   False,
                          'grid_search':                   False}
            with open(self.LOGDIR + 'todo_list.json', 'w') as file: json.dump(dictionary, file)
        return dictionary
    
    def _feature_engineering(self, dictionary):
        feature_engineering_file_url = self.LOGDIR + 'feature_engineering.txt'
        if not dictionary['feature_engineering']:
            if not os.path.exists(feature_engineering_file_url):
                self._generate_feature_engineering_file(feature_engineering_file_url)
            with open(feature_engineering_file_url, 'r') as file:
                line = file.readline()
                while line:
                    line = re.sub('\\n', '', line)
                    self._add_column(line)
                    # TODO
                    # feature engineering: create feature line by line in the file
        self._renew_status(dictionary, 'feature_engineering', (self.LOGDIR + 'todo_list.json'))
    
    def _validation_and_down_sampling(self, dictionary):
        split_folder = []
        index = []
        down_sampling_url = None
        if not dictionary['val_downsample_generation']:
            # down sampling
            down_sampling_url = self.DATADIR + 'split/'
            if not os.path.exists(down_sampling_url): os.makedirs(down_sampling_url)
            with open(down_sampling_log_url, 'r') as file: downsampling_work_log = json.load(file)
            index.extend(self._generate_down_sampling_index_file(dictionary['val_downsample_generate_index']))
            for i in range(self.downsampling): split_folder.append(down_sampling_url+str(i)+'/')

            # validation
            split_folder.append(self.DATADIR + 'split/val/')
            index.append(self.validation_index)
            if not os.path.exists(split_folder[-1]): os.makedirs(split_folder[-1])
                
            self._renew_status(dictionary, 'val_downsample_generate_index', self.LOGDIR + 'todo_list.json')
        
        # split files
        if not dictionary['val_downsample_split']:
            for file in os.path.listdirs(self.FEATUREDIR):
                split_file = file.split('.')
                if split_file[-1] == 'ftr':
                    splitted = split_file[0].split('__')
                    if splitted[0] == 'train':
                        for i in range(len(index)):
                            
                
            self._renew_status(dictionary, 'val_downsample_split', self.LOGDIR + 'todo_list.json')
        
        # concat files
        if not dictionary['val_downsample_generation']:
            if down_sampling_url == None:
                index.append(sorted(list(set(range(self.train_len)).difference(set(self.validation_index)))))
                split_folder.append(self.FEATUREDIR)
            for i in range(len(split_folder)):
                file_name_body = 'val' if split_folder[i] == self.FEATUREDIR else split_folder[i].split('/')[-1]
                self._get_file_concat(split_folder=split_folder[i], 
                                      concat_folder=self.DATADIR, 
                                      is_train=True, 
                                      file_name_body=file_name_body, 
                                      useindex=index[i])
            self._renew_status(dictionary, 'val_downsample_generation', self.LOGDIR + 'todo_list.json')
        
        
#         # split file
#             for i in range(self.downsampling):
#                 if not downsampling_work_log[str(i)]:
#                     if not os.path.exists(down_sampling_url+str(i)+'/'): os.makedirs(down_sampling_url + str(i) + '/')
#                     self._get_file_split(index_list[i], down_sampling_url + str(i) + '/')
#                     self._renew_status(downsampling_work_log, str(i), self.LOGDIR + 'downsampling.json')
                    
#         # concatenant file
#         for i in range(self.downsampling):
#             if not downsampling_work_log['concat' + str(i)]:
#                 self._get_file_concat(down_sampling_url + str(i) + '/', down_sampling_url)
#                 self._renew_status(downsampling_work_log, 'concat' + str(i), self.LOGDIR + 'downsampling.json')
#             self._renew_status(dictionary, 'down_sampling', self.LOGDIR + 'todo_list.json')
    
#     def _down_sampling(self, dictionary):
#         down_sampling_log_url = self.LOGDIR + 'downsampling.json'
#         down_sampling_url = self.DATADIR + 'downsampling/'
#         if not dictionary['down_sampling']:
#             if not os.path.exists(down_sampling_url): os.makedirs(down_sampling_url)
#             if not os.path.exists(down_sampling_log_url): self._generate_down_sampling_file(down_sampling_log_url)
#             with open(down_sampling_log_url, 'r') as file: downsampling_work_log = json.load(file)
#             index_list = self._generate_down_sampling_index_file(downsampling_work_log['generate_index_file'])
#             self._renew_status(downsampling_work_log, 'generate_index_file', self.LOGDIR + 'downsampling.json')
            
#             # split file
#             for i in range(self.downsampling):
#                 if not downsampling_work_log[str(i)]:
#                     if not os.path.exists(down_sampling_url+str(i)+'/'): os.makedirs(down_sampling_url + str(i) + '/')
#                     self._get_file_split(index_list[i], down_sampling_url + str(i) + '/')
#                     self._renew_status(downsampling_work_log, str(i), self.LOGDIR + 'downsampling.json')
                    
#             # concatenant file
#             for i in range(self.downsampling):
#                 if not downsampling_work_log['concat' + str(i)]:
#                     self._get_file_concat(down_sampling_url + str(i) + '/', down_sampling_url)
#                     self._renew_status(downsampling_work_log, 'concat' + str(i), self.LOGDIR + 'downsampling.json')
                    
#             self._renew_status(dictionary, 'down_sampling', self.LOGDIR + 'todo_list.json')
    
#     def _generate_validation(self, dictionary):
#         # TODO: get validation set and save to ftr with validation index given
        
#         self._renew_status(dictionary, 'validation_generation', self.LOGDIR + 'todo_list.json')
    
    def _grid_search(self, dictionary):
        # TODO: finish grid search
        self._renew_status(dictionary, 'grid_search', self.LOGDIR + 'todo_list.json')
    
    
    ######### support functions #########
    # feature engineering
    def _generate_feature_engineering_file(self, feature_engineering_file_url):
        with open(feature_engineering_file_url, 'w') as file:
            pass
    
    # create a feature
    def _add_column(self):
        pass
    
    # concat test
    def _concat_test(self, dictionary):
        self._get_file_concat(split_folder=self.FEATUREDIR,
                              concat_folder=self.DATADIR,
                              is_train=False,
                              file_name_body='all_features')
        self._renew_status(dictionary, 'concat_test', self.LOGDIR + 'todo_list.json')
        
#     # down sampling
#     def _generate_down_sampling_file(self, down_sampling_file_url):
#         dictionary = {'generate_index_file': False}
#         for i in range(self.downsampling):
#             dictionary[str(i)] = False
#             dictionary['concat' + str(i)] = False
#         with open(down_sampling_file_url, 'w') as file: json.dump(dictionary, file)
    
    def _generate_down_sampling_index_file(self, has_file_built):
        return [[1,2,3]]
    
#     def _get_file_split(self, index, split_folder):
#         pass
    
    # grid search
    def _generate_grid_search_file(self):
        pass
    
    """ 
    feature_engineering todo list
    feature_engineering.txt line: <function_name>__<feature_combination_name>__<possible_param>
    file_name: train__<function_name>__<feature_combination_name>__<possible_param>.ftr
                test__<function_name>__<feature_combination_name>__<possible_param>.ftr
    """

    """
    down sampling todo list
    if positive : negative < 1 : 1 in train (validation excluded), do down sampling, sample 5 files in default
    """
#     train_exclude_val = self.train.drop(self.validation_index, axis=0)
#     positive = list(train_exclude_val[train_exclude_val[self.label==1]].index.values)
#     negative = list(train_exclude_val[train_exclude_val[self.label==0]].index.values)
#     ratio = len(positive) / len(negative)
#     if ratio >= 0.5:
#         dictionary['down_sampling'] = True
#     else:
#         down_sampling_dict = {}

In [3]:
# a = AlphaBoosting(configuration='./log/config.json')
a = AlphaBoosting(root='./')

In [28]:
x = [[1,2]]
y = list(x)
y[0][0] = 3
print(x)

[[3, 2]]
